## Use a python 3.11 kernel cehrbert

In [11]:
!pip install meds_reader==0.1.9
!pip install setuptools
!pip install cehrbert==1.3.1

  Using cached meds-0.3.3-py3-none-any.whl.metadata (10 kB)
  Using cached pandas-2.2.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
  Using cached jsonschema-4.23.0-py3-none-any.whl.metadata (7.9 kB)
  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached attrs-24.2.0-py3-none-any.whl.metadata (11 kB)
  Using cached jsonschema_specifications-2024.10.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached referencing-0.35.1-py3-none-any.whl.metadata (2.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 12.8 MB/s eta 0:00:00 0:00:01
Using cached meds-0.3.3-py3-none-any.whl (12 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 87.9 MB/s eta 0:00:00:00:01
Using cached pandas-2.2.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.1 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 MB 143.8 MB/s eta 0:00:0000:01
Using cached jsonsc

In [12]:
#@title Download E-ICU demo
import os
from pathlib import Path
notebook_dir = os.getcwd()

ROOT_DIR=f"{notebook_dir}/work_dir/mimiciv_demo/"
# ROOT_DIR=f"{notebook_dir}/work_dir/eicu_demo/"
Path(ROOT_DIR).mkdir(parents=True, exist_ok=True)

In [ ]:
MEDS_DIR = ROOT_DIR + "/meds/"
MEDS_READER_DIR = ROOT_DIR + "/meds_reader/"
TASK_DIR = MEDS_DIR + "/task_labels"
TASK_NAME="mortality/in_icu/first_24h"
# TASK_NAME="los_in_hospital_first_48h"
OUTPUT_PRETRAIN_MODEL_DIR= ROOT_DIR + "/output/cehrbert/train/"
# TODO this variable has an identical name?
OUTPUT_FINETUNE_MODEL_DIR= ROOT_DIR + "/output/cehrbert/finetuned/"

Run meds_reader on the MEDS data

In [ ]:
!meds_reader_convert $MEDS_DIR $MEDS_READER_DIR

In [15]:
!mkdir -p {ROOT_DIR}/output/cehrbert/
!mkdir -p {ROOT_DIR}/output/cehrbert_dataset_prepared/
!mkdir -p {ROOT_DIR}/output/cehrbert_finetuned/

In [16]:
!mkdir {ROOT_DIR}/github_repo;cd {ROOT_DIR}/github_repo;git clone https://github.com/cumc-dbmi/cehrbert.git;cd cehrbert;git checkout fix/meds_evaluation;pip install .;

Cloning into 'cehrbert'...
remote: Enumerating objects: 8757, done.
remote: Counting objects: 100% (1428/1428), done.
remote: Compressing objects: 100% (604/604), done.
remote: Total 8757 (delta 934), reused 943 (delta 767), pack-reused 7329 (from 1)
Receiving objects: 100% (8757/8757), 14.23 MiB | 25.66 MiB/s, done.
Resolving deltas: 100% (6174/6174), done.
error: pathspec 'fix/meds_evaluation' did not match any file(s) known to git
Processing /storage/nassim/projects/MEDS-DEV/demo/work_dir/mimiciv_demo/github_repo/cehrbert
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for cehrbert: filename=cehrbert-1.3.1-py3-none-any.whl size=139740 sha256=b67dbe857d4ecf2d4c1c7b2fad5ece256e911d33abef142535a1210274be4205
  Stored in directory: /tmp/pip-ephem-wheel-cache-j1feuvmd/wheels/0f/e9/2b/6ce226fd4943719e4d11e237506934d50e642553b2941d21c5
Successfully built cehrbert
  Attempting uninstall: c

Create the cehrbert pretraining configuration yaml file

In [17]:
cehrbert_pretrain_config = f"""
#Model arguments
model_name_or_path: "{ROOT_DIR}/output/cehrbert/"
tokenizer_name_or_path: "{ROOT_DIR}/output/cehrbert/"
num_hidden_layers: 6
max_position_embeddings: 1024
hidden_size: 768
vocab_size: 100000
min_frequency: 50
include_value_prediction: false # additional CEHR-BERT learning objective

#Data arguments
data_folder: "{ROOT_DIR}/meds_reader/"
dataset_prepared_path: "{ROOT_DIR}/output/cehrbert_dataset_prepared/"

# Below is a list of Med-to-CehrBert related arguments
preprocessing_num_workers: 2
preprocessing_batch_size: 128
# if is_data_in_med is false, it assumes the data is in the cehrbert format
is_data_in_meds: true
att_function_type: "cehr_bert"
inpatient_att_function_type: "mix"
include_auxiliary_token: true
include_demographic_prompt: false
# if the data is in the meds format, the validation split will be omitted
# as the meds already provide train/tuning/held_out splits
validation_split_percentage: 0.05

# Huggingface Arguments
dataloader_num_workers: 2
dataloader_prefetch_factor: 2

overwrite_output_dir: false
resume_from_checkpoint: # automatically infer the latest checkpoint from the output folder
seed: 42

output_dir: "{ROOT_DIR}/output/cehrbert/"
evaluation_strategy: "epoch"
save_strategy: "epoch"
eval_accumulation_steps: 10

learning_rate: 0.00005
per_device_train_batch_size: 8
per_device_eval_batch_size: 8
gradient_accumulation_steps: 2

num_train_epochs: 50 # for large datasets, 5-10 epochs should suffice
warmup_steps: 10
weight_decay: 0.01
logging_dir: "./logs"
logging_steps: 10

save_total_limit:
load_best_model_at_end: true
metric_for_best_model: "eval_loss"
greater_is_better: false

report_to: "none"
"""
PRETRAIN_CONFIG_FP = ROOT_DIR + "/output/cehrbert/cehrbert_pretrain_config.yaml"
with open(PRETRAIN_CONFIG_FP, 'w') as f:
    f.write(cehrbert_pretrain_config)

In [19]:
## Pretrain cehrbert using MLM
!python3.11 -m cehrbert.runners.hf_cehrbert_pretrain_runner {ROOT_DIR}/output/cehrbert/cehrbert_pretrain_config.yaml

2024-12-15 03:56:01.498490: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-15 03:56:01.498535: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-15 03:56:01.499647: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-15 03:56:01.506092: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-15 03:56:02.465497: W tensorflow/comp

In [27]:
## Create the cehrbert finetuning configuration yaml file
cehrbert_finetune_config = f"""
#Model arguments
model_name_or_path: "{ROOT_DIR}/output/cehrbert/"
tokenizer_name_or_path: "{ROOT_DIR}/output/cehrbert/"
num_hidden_layers: 6
max_position_embeddings: 1024
hidden_size: 768
vocab_size: 100000
min_frequency: 50
include_value_prediction: false # additional CEHR-BERT learning objective

#Data arguments
cohort_folder: "{TASK_DIR}/{TASK_NAME}/"
data_folder: "{ROOT_DIR}/meds_reader/"
dataset_prepared_path: "{ROOT_DIR}/output/cehrbert_dataset_prepared/"

#LORA
use_lora: True
lora_rank: 64
lora_alpha: 16
target_modules: [ "query", "value" ]
lora_dropout: 0.1

#Below is a list of Med-to-CehrBert related arguments
preprocessing_num_workers: 2
preprocessing_batch_size: 128
#if is_data_in_med is false, it assumes the data is in the cehrbert format
is_data_in_meds: true
att_function_type: "cehr_bert"
inpatient_att_function_type: "mix"
include_auxiliary_token: true
include_demographic_prompt: false
#if the data is in the meds format, the validation split will be omitted
#as the meds already provide train/tuning/held_out splits
validation_split_percentage: 0.05

#Huggingface Arguments
dataloader_num_workers: 2
dataloader_prefetch_factor: 2

overwrite_output_dir: false
resume_from_checkpoint: # automatically infer the latest checkpoint from the output folder
seed: 42

output_dir: "{ROOT_DIR}/output/cehrbert_finetuned"
evaluation_strategy: "epoch"
save_strategy: "epoch"
eval_accumulation_steps: 10

do_train: True
do_predict: True

learning_rate: 0.00005
per_device_train_batch_size: 8
per_device_eval_batch_size: 8
gradient_accumulation_steps: 2

num_train_epochs: 10
warmup_steps: 10
weight_decay: 0.01
logging_dir: "./logs"
logging_steps: 10

save_total_limit:
load_best_model_at_end: true
metric_for_best_model: "eval_loss"
greater_is_better: false

report_to: "none"
"""
FINETUNE_CONFIG_FP = f"{ROOT_DIR}/output/cehrbert/cehrbert_finetune_config.yaml"
with open(FINETUNE_CONFIG_FP, 'w') as f:
    f.write(cehrbert_finetune_config)

In [28]:
# ## Finetune cehrbert for the downstream task
!python3.11 -m cehrbert.runners.hf_cehrbert_finetune_runner {ROOT_DIR}/output/cehrbert/cehrbert_finetune_config.yaml

2024-12-15 04:10:54.603602: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-15 04:10:54.603651: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-15 04:10:54.604733: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-15 04:10:54.611030: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-15 04:10:55.415341: W tensorflow/comp

In [31]:
!cat {ROOT_DIR}/output/cehrbert_finetuned/test_results.json

{
    "roc_auc": 0.4,
    "pr_auc": 0.05,
    "test_loss": 0.6830134987831116
}